# Transfer Learning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import cv2
import os
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential

In [2]:
# ONLY FOR NVIDIA
CUDA_VISIBLE_DEVICES = 0

In [3]:
tf.config.list_physical_devices('GPU')

2024-08-23 00:13:11.386217: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


[]

In [4]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, cache_dir='.', untar=True)

In [5]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [6]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [7]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [8]:
flowers_images_dict['roses'][:5]

[PosixPath('datasets/flower_photos/roses/10503217854_e66a804309.jpg'),
 PosixPath('datasets/flower_photos/roses/19823402005_2db025dd66_m.jpg'),
 PosixPath('datasets/flower_photos/roses/12240577184_b0de0e53ea_n.jpg'),
 PosixPath('datasets/flower_photos/roses/563847503_89e9756c80.jpg'),
 PosixPath('datasets/flower_photos/roses/3829990289_c0c3821e4d_m.jpg')]

In [9]:
def resize_images(df):
    X, y = [], []
    for flower_name, images in flowers_images_dict.items():
        for image in images:
            img = cv2.imread(str(image))
            resized_img = cv2.resize(img, (224,224))
            X.append(resized_img)
            y.append(flowers_labels_dict[flower_name])
    return X, y

In [10]:
X, y = resize_images(flowers_images_dict)

In [11]:
X = np.array(X)
y = np.array(y)

# Scale/split Data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [14]:
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

2752 918
2752 918


In [15]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [16]:
IMAGE_SHAPE = (244,244)
IMAGE_SHAPE+(3,)


(244, 244, 3)

# ANN Model

In [17]:
from sklearn.metrics import classification_report, confusion_matrix

In [18]:
ann = Sequential([
    Flatten(),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='softmax')
])
ann.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [19]:
ann.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [20]:
ann.fit(X_train, y_train, epochs=5)

Epoch 1/5


2024-08-23 00:02:10.171580: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1657012224 exceeds 10% of free system memory.


86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 155ms/step - accuracy: 0.2437 - loss: 13.4784
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 156ms/step - accuracy: 0.3938 - loss: 2.7640
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 159ms/step - accuracy: 0.4461 - loss: 2.4973
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 13s 155ms/step - accuracy: 0.4354 - loss: 2.8439
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 13s 156ms/step - accuracy: 0.4722 - loss: 2.1825


In [22]:
y_pred = ann.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]
print(classification_report(y_test, y_pred_classes))

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
              precision    recall  f1-score   support

           0       0.30      0.80      0.44       176
           1       0.47      0.15      0.23       154
           2       0.49      0.15      0.23       226
           3       0.39      0.80      0.52       150
           4       0.36      0.04      0.08       212

    accuracy                           0.36       918
   macro avg       0.40      0.39      0.30       918
weighted avg       0.40      0.36      0.28       918



# Transfer Learning

In [35]:
print(hub.version)

<module 'tensorflow_hub.version' from '/home/shawn/anaconda3/lib/python3.12/site-packages/tensorflow_hub/version.py'>


In [26]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import tf_keras

In [28]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224,224,3), trainable=False, 
)

In [29]:
num_of_flowers = 5

model = Sequential([
    pretrained_model_without_top_layer,
    Dense(num_of_flowers, activation='softmax')
])

model.summary()

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x716558167170> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [21]:
model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [22]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5


2024-08-23 00:13:25.062711: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1657012224 exceeds 10% of free system memory.


48/86 ━━━━━━━━━━━━━━━━━━━━ 1:52 3s/step - accuracy: 0.2765 - loss: 2.2514

KeyboardInterrupt: 

In [27]:
X_train.shape

(2752, 224, 224, 3)